In [102]:
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import torch
import torch_sparse

OSError: libcudart.so.10.2: cannot open shared object file: No such file or directory

**Node Embeddings**

In [6]:
emb = np.load("../IllinoisGraphDataset_v1/paper_384_emb_1.npy", allow_pickle=True)

In [13]:
paper_ids_with_emb = set()

In [14]:
for paper_id, paper_emb in emb.item().items():
    paper_ids_with_emb.add(paper_id)

**Edges**

In [2]:
edges = np.load("../IllinoisGraphDataset_v1/cites_edge_1.npy", mmap_mode='r')

In [23]:
required_edges = []

In [26]:
count = 0
for edge in tqdm(edges):
    if edge[0] in common_paper_ids and edge[1] in common_paper_ids:
        required_edges.append(edge)
    count += 1

100%|██████████| 399999999/399999999 [31:46<00:00, 209847.61it/s]


In [27]:
count

399999999

In [28]:
len(required_edges)

915124

In [30]:
# get the most connected nodes
node_connectivity = defaultdict(int)
for edges in required_edges:
    node_connectivity[edges[0]] += 1
    node_connectivity[edges[1]] += 1

In [34]:
node_connectivity = {k: v for k, v in sorted(node_connectivity.items(), key=lambda item: item[1], reverse=True)}

In [35]:
node_connectivity

{2144634347: 1822,
 1987258130: 374,
 2167590372: 371,
 2124776405: 363,
 2109469951: 361,
 2108795964: 356,
 2125055259: 347,
 2071666535: 341,
 2142635246: 325,
 2125121305: 292,
 2124758339: 285,
 1539796472: 281,
 2108818539: 263,
 2059334100: 235,
 2061901927: 233,
 2028622989: 230,
 2166559705: 226,
 2109911863: 223,
 2020999234: 218,
 2124614417: 213,
 2142183404: 213,
 1976499671: 197,
 1550621568: 195,
 2020830322: 182,
 2007395042: 175,
 1540380506: 172,
 2125435699: 170,
 2028231353: 169,
 2141846678: 165,
 2022950330: 164,
 1541515907: 163,
 2109401564: 162,
 2109363337: 161,
 1541782411: 161,
 2147152072: 158,
 2166974667: 156,
 2028770837: 155,
 2144357229: 154,
 2166785990: 151,
 1597750586: 145,
 2166858912: 144,
 2146203649: 143,
 1551024839: 136,
 2142596760: 135,
 2142175015: 133,
 1541434994: 128,
 2141885858: 125,
 2007221293: 124,
 74492904: 123,
 2142644281: 119,
 2142063750: 119,
 1498436455: 118,
 2146396346: 117,
 1560013842: 115,
 2109671377: 109,
 1597773817

In [36]:
most_connected_nodes = dict(filter(lambda elem: elem[1] > 10 , node_connectivity.items()))

In [38]:
len(most_connected_nodes)

11448

**Lables**

In [8]:
labels = np.load("../IllinoisGraphDataset_v1/paper_ss_19_labels.npy", allow_pickle=True)

**Common `paper_ids` between the labels and embeddings**

In [15]:
paper_ids_with_labels = set()

In [16]:
for paper_id, paper_label in labels.item().items():
    paper_ids_with_labels.add(paper_id)

In [17]:
len(paper_ids_with_emb)

10000384

In [18]:
len(paper_ids_with_labels)

157675969

In [19]:
#common_paper_ids = paper_ids_with_labels.intersection(paper_ids_with_emb)

In [21]:
len(common_paper_ids)

6395200

**Creating a `11448` node graph**

In [69]:
citation_edges = set(most_connected_nodes.keys())

In [91]:
paper_node_edges = [[], []]
for edg in required_edges:
    if edg[0] in citation_edges or edg[1] in citation_edges:
        paper_node_edges[0].append(edg[0])
        paper_node_edges[1].append(edg[1])
#         paper_node_edges[0].append(edg[1])
#         paper_node_edges[1].append(edg[0])

In [92]:
v = [1 for _ in range(len(paper_node_edges[0]))]

In [93]:
adj = torch.sparse_coo_tensor(indices=paper_node_edges, values=v).coalesce()

In [95]:
sum(most_connected_nodes.values())

208474

In [96]:
len(paper_node_edges[0])

197944